# Imports and Constants

An API key must be given. Registration required first.

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import requests
import pandas as pd
import numpy as np

from datetime import datetime
from dateutil import parser

from dotenv import load_dotenv

from tqdm.notebook import tqdm

# import metalhistory as mh
import metalhistory.data_query_functions as dqf

import requests
import pandas as pd
import xmltodict
import time

In [3]:
tqdm.pandas()

# Data Preprocessing

In [4]:
%%time
df_csv = pd.read_csv('data/MA_10k_albums.csv')
df_csv

CPU times: user 7.78 ms, sys: 0 ns, total: 7.78 ms
Wall time: 7.3 ms


,artist,album,MA_score
0,Slayer,Reign in Blood,36.01
1,Metallica,Kill 'Em All,33.39
2,Hades Archer,Penis Metal,32.67
3,Iron Maiden,Iron Maiden,32.38
4,Metallica,Master of Puppets,31.83
...,...,...,...
9995,Iron Maiden,Live at the Rainbow,1.92
9996,Jorn,Worldchanger,1.92
9997,Juggernaut,Trouble Within,1.92
9998,Lacrimas Profundere,Memorandum,1.92


# Alternative
Don't use album matches! Because they always crash after a while!

In [6]:
%%time
from IPython.display import clear_output
lastfm = dqf.LastFM()

HEADS = np.arange(100, 5000, 100)
TAIL = 100

for HEAD in HEADS:
    clear_output(wait=True)
    result_df = df_csv.head(HEAD).tail(TAIL)

    fields =['artist', 'name', 'release-date', 'listeners', 'playcount', 'tags', 'mbid', 'url', 'image']

    result_df['lastfm_info'] = result_df.progress_apply(lambda row: lastfm.get_album_info(artist=row['artist'], album=row['album'], fields=fields), axis=1)

    result_df= result_df.lastfm_info.apply(pd.Series)
    result_df['MA_score'] = df_csv.head(HEAD).tail(TAIL)['MA_score']
    result_df['MA_artist'] = df_csv.head(HEAD).tail(TAIL)['artist']
    result_df['MA_album'] = df_csv.head(HEAD).tail(TAIL)['album']

    fields.append('ignored tags')
    result_df.rename(columns={'name':'album'}, inplace=True)
    result_df = result_df.set_index(['artist', 'album'])


    show_fields =['release-date', 'listeners', 'playcount', 'tags', 'ignored tags']
    result_df.sort_values(by='listeners', ascending=False).to_csv('./data/proc_MA_'+ str(HEAD)+ '_albums_non_cumulative.csv')

  0%|          | 0/50 [00:00<?, ?it/s]

CPU times: user 1.06 s, sys: 43.4 ms, total: 1.11 s
Wall time: 30 s
